In [1]:
import wmfdata as wmf

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [22]:
mob_wikis = (
    "hiwiki",
    "bnwiki",
    "idwiki",
    "arwiki",
    "mrwiki",
    "fawiki",
    "swwiki",
    "tlwiki",
    "zhwikiquote",
    "thwiki",
    "arzwiki",
    "mlwiki",
    "tawiki",
    "knwiki",
    "ptwiktionary",
    "azwiki",
    "guwiki",
    "kywiki",
    "sqwiki",
    "mswiki"
)

gn_countries = (
    "AD", "AL", "AT", "AX", "BA", "BE", "BG", "CH", "CY", "CZ",
    "DE", "DK", "EE", "ES", "FI", "FO", "FR", "FX", "GB", "GG",
    "GI", "GL", "GR", "HR", "HU", "IE", "IL", "IM", "IS", "IT",
    "JE", "LI", "LU", "LV", "MC", "MD", "ME", "MK", "MT", "NL",
    "NO", "PL", "PT", "RO", "RS", "RU", "SE", "SI", "SJ", "SK",
    "SM", "TR", "VA", "AU", "CA", "HK", "MO", "NZ", "JP", "SG",
    "KR", "TW", "US"
)

# Global South countries

In [ ]:
gs_edits = wmf.hive.run("""
with gs_editors as (
    select 
        wiki_db,
        user_fingerprint_or_id as user_id
    from wmf.geoeditors_daily
    where
        month >= "{start}" and
        country_code not in {gn_countries} and
        user_is_anonymous = 0
    group by wiki_db, user_fingerprint_or_id
)
select
    date_format(event_timestamp, "yyyy-MM-01") as month,
    count(*) as total_edits,
    sum(cast(
        !(event_user_is_bot_by_name or array_contains(event_user_groups, "bot")
    ) as int)) as nonbot_edits
from gs_editors ge
left join wmf.mediawiki_history mh
on
    ge.wiki_db = mh.wiki_db and
    ge.user_id = event_user_id and
    snapshot = "{snapshot}"
where
    event_entity = "revision" and
    event_type = "create" and
    event_timestamp >= "{start}"
group by date_format(event_timestamp, "yyyy-MM-01")
""".format(
    gn_countries=repr(gn_countries),
    snapshot="2018-06",
    start="2018-04"
))

In [32]:
gs_edits

,month,total_edits,nonbot_edits
0,2018-04-01,8524266,8524226
1,2018-05-01,9227996,9227994
2,2018-06-01,8514315,8514314


In [42]:
gs_editors = wmf.hive.run("""
with gs_editors as (
    select 
        wiki_db,
        user_fingerprint_or_id as user_id
    from wmf.geoeditors_daily
    where
        month >= "{start}" and
        country_code not in {gn_countries} and
        user_is_anonymous = 0
    group by wiki_db, user_fingerprint_or_id
)
select
    month,
    sum(cast(content_edits >= 5 as int)) as active_editors
from (
    select
        date_format(event_timestamp, "yyyy-MM-01") as month,
        count(*) as content_edits
    from gs_editors ge
    left join wmf.mediawiki_history mh
    on
        ge.wiki_db = mh.wiki_db and
        ge.user_id = event_user_id and
        snapshot = "{snapshot}"
    where
        event_entity = "revision" and
        event_type = "create" and
        event_timestamp >= "{start}" and
        page_namespace_is_content = 1 and
        !(event_user_is_bot_by_name or array_contains(event_user_groups, "bot"))
    group by event_user_text, date_format(event_timestamp, "yyyy-MM-01")
) combined_eds
group by month
""".format(
    gn_countries=repr(gn_countries),
    snapshot="2018-06",
    start="2018-04"
))

In [45]:
gs_editors

,month,active_editors
0,2018-04-01,21606
1,2018-05-01,22164
2,2018-06-01,21490


In [57]:
gs_ner = wmf.hive.run("""
with gs_edits as (
    select 
        gd.wiki_db,
        event_user_text as user_name,
        event_timestamp as edit_dt,
        event_user_creation_timestamp as registration_dt
    from wmf.geoeditors_daily gd
    left join wmf.mediawiki_history mh
    on
        gd.wiki_db = mh.wiki_db and
        gd.user_fingerprint_or_id = event_user_id and
        snapshot = "{snapshot}"
    where
        month >= "{start}" and
        country_code not in {gn_countries} and
        gd.user_is_anonymous = 0 and
        event_entity = "revision" and
        event_type = "create" and
        event_timestamp >= "{start}" and
        event_user_is_created_by_system = 0 and
        event_user_creation_timestamp >= "{start}" and
        event_user_creation_timestamp < "{end}" and
        not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot"))
)
select 
    1st_month.cohort,
    sum(cast(1st_month.edits >= 1 as int)) as new_editors,
    sum(cast(2nd_month.edits >= 1 as int)) / sum(cast(1st_month.edits >= 1 as int)) as new_editor_retention
from (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from gs_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60))
    group by user_name, registration_dt, wiki_db
    ) 1st_month
left join (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from gs_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") >=
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60)) and
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (60*24*60*60))
        group by user_name, registration_dt, wiki_db
    ) 2nd_month
on
    (1st_month.user_name = 2nd_month.user_name and
    1st_month.wiki = 2nd_month.wiki and
    1st_month.cohort = 2nd_month.cohort)
group by 1st_month.cohort
""".format(
    gn_countries=repr(gn_countries),
    snapshot="2018-06",
    start="2018-04",
    end="2018-05"
))

In [58]:
gs_ner

,1st_month.cohort,new_editors,new_editor_retention
0,2018-04,37177,0.044275


# Mobile-heavy wikis

In [60]:
mh_edits = wmf.hive.run("""
select
    date_format(event_timestamp, "yyyy-MM-01") as month,
    count(*) as total_edits,
    sum(cast(not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot")) as int)) as nonbot_edits
from wmf.mediawiki_history
where
    snapshot = "{snapshot}" and
    event_entity = "revision" and
    event_type = "create" and
    wiki_db in {wikis} and
    event_timestamp >= "{start}" and
    event_timestamp < "{end}" and
    not event_user_is_anonymous
group by date_format(event_timestamp, "yyyy-MM-01")
""".format(
    snapshot="2018-06",
    start="2017-06",
    end="2018-07",
    wikis=repr(mob_wikis)
))

In [67]:
mh_edits.sort_values("month")

,month,total_edits,nonbot_edits
0,2017-06-01,738181,423084
7,2017-07-01,944735,459680
1,2017-08-01,1039919,440710
8,2017-09-01,1242239,413032
2,2017-10-01,1045400,412664
9,2017-11-01,895943,438824
3,2017-12-01,1088167,431783
10,2018-01-01,1266697,465025
4,2018-02-01,865871,425375
11,2018-03-01,1098456,447971


In [66]:
mh_editors = wmf.hive.run("""
select
    month,
    sum(cast(content_edits >= 5 as int)) as active_editors
from (
    select
        date_format(event_timestamp, "yyyy-MM-01") as month,
        count(*) as content_edits
    from wmf.mediawiki_history
    where
        snapshot = "{snapshot}" and
        event_entity = "revision" and
        event_type = "create" and
        wiki_db in {wikis} and
        event_timestamp >= "{start}" and
        event_timestamp < "{end}" and
        not event_user_is_anonymous and
        not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot")) and
        page_namespace_is_content
    group by date_format(event_timestamp, "yyyy-MM-01"), event_user_text
) editors
group by month
""".format(
    snapshot="2018-06",
    start="2017-06",
    end="2018-07",
    wikis=repr(mob_wikis)
))

In [69]:
mh_editors.sort_values("month")

,month,active_editors
0,2017-06-01,3571
7,2017-07-01,4096
1,2017-08-01,3430
8,2017-09-01,3211
2,2017-10-01,3350
9,2017-11-01,3330
3,2017-12-01,3524
10,2018-01-01,3807
4,2018-02-01,3600
11,2018-03-01,3641


In [92]:
mh_ner = wmf.hive.run("""
with mh_edits as (
    select 
        wiki_db,
        event_user_text as user_name,
        event_timestamp as edit_dt,
        event_user_creation_timestamp as registration_dt
    from wmf.mediawiki_history        
    where
        snapshot = "{snapshot}" and
        event_entity = "revision" and
        event_type = "create" and
        event_timestamp >= "{start}" and
        wiki_db in {wikis} and
        not event_user_is_anonymous and
        not event_user_is_created_by_system and
        event_user_creation_timestamp >= "{start}" and
        event_user_creation_timestamp < "{end}" and
        not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot"))
)
select 
    1st_month.cohort,
    sum(cast(1st_month.edits >= 1 as int)) as new_editors,
    sum(cast(2nd_month.edits >= 1 as int)) / sum(cast(1st_month.edits >= 1 as int)) as new_editor_retention
from (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from mh_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60))
    group by user_name, registration_dt, wiki_db
    ) 1st_month
left join (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from mh_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") >=
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60)) and
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (60*24*60*60))
        group by user_name, registration_dt, wiki_db
    ) 2nd_month
on
    (1st_month.user_name = 2nd_month.user_name and
    1st_month.wiki = 2nd_month.wiki and
    1st_month.cohort = 2nd_month.cohort)
group by 1st_month.cohort
""".format(
    snapshot="2018-06",
    start="2017-04",
    end="2018-05",
    wikis=repr(mob_wikis)
))

In [93]:
mh_ner

,1st_month.cohort,new_editors,new_editor_retention
0,2017-04,8227,0.042907
1,2017-05,7531,0.061346
2,2017-06,7390,0.052233
3,2017-07,7837,0.049381
4,2017-08,7617,0.042405
5,2017-09,7363,0.049301
6,2017-10,8550,0.044444
7,2017-11,8032,0.042953
8,2017-12,8298,0.043022
9,2018-01,8693,0.044173


In [95]:
ner = mh_ner["new_editor_retention"]
(ner[12] - ner[0]) / ner[0]

-0.054497600196141645